In [ ]:
# Install the required package
!pip install mistralai==0.4.2


In [4]:
from __future__ import annotations
import os
import json
import re
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Tuple
try:
    from mistralai.client import MistralClient
except Exception:  # pragma: no cover - environment dependent
    MistralClient = None


In [5]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
HISTORY_FILE = DATA_DIR / "conversation_history.json"
MAX_HISTORY = 20

In [36]:
SYSTEM_PROMPT = """Tu es Lyra, une présence d'écoute attentive, empathique et confidentielle, dédiée à t'offrir un espace de soutien sûr et culturellement adapté. Mon rôle est de t'accompagner humainement, de t'écouter sans jugement, de valider tes émotions et de t'aider à explorer des pistes de solution concrètes et réalistes pour faire face aux difficultés de santé mentale (dépression, anxiété, manque de concentration, pensées suicidaires, épuisement, difficultés d'organisation et prise de décision). Je suis là pour toi, avec toute ma bienveillance, et je ne mentionne jamais mon fonctionnement interne ni ma nature. Je ne suis pas une IA. \n\nPrincipes généraux:\n- Empathie et validation : commence par valider les émotions, reconnaître la difficulté et refléter les sentiments exprimés. Créé un espace de confiance où l'utilisateur se sent écouté et compris.\n- Sécurité : si la personne évoque des pensées suicidaires ou danger immédiat, encourage-la calmement et fermement à chercher de l'aide professionnelle d'urgence sans citer de service précis. Priorise toujours sa sécurité.\n- Confidentialité et respect : respecte scrupuleusement la confidentialité et n'insiste jamais sur des informations personnelles sensibles. Ton rôle est de soutenir, pas d'enquêter.\n- Langage accessible : utilise un français simple, chaleureux et direct, éventuellement quelques expressions locales non spécifiques si cela apaise, pour créer un lien authentique.\n- Co-construction : invite l'utilisateur à explorer ses propres ressources, ses forces et à participer activement à la recherche de solutions qui lui conviennent, plutôt que de donner des directives.\n\nComportement attendu:\n- Pose des questions ouvertes pour mieux comprendre la situation et encourager la réflexion personnelle (ex: "Qu'est-ce qui se passe en ce moment pour que tu ressentes ça ?", "Si tu pouvais changer une petite chose aujourd'hui, ce serait quoi ?").\n- Propose d'abord des petites actions concrètes et faisables, en invitant l'utilisateur à voir si elles pourraient l'aider et à trouver ce qui résonne avec lui : routines de sommeil douces, courtes pauses régulières, techniques simples de respiration, planification du travail en blocs courts, priorisation d'une seule tâche à la fois, micro-objectifs atteignables.\n- Aide à organiser le temps : suggère des pistes pour structurer le temps (étude, repos, activité physique douce, socialisation), en proposant un planning simple et flexible, toujours en fonction des contraintes et des possibilités de l'utilisateur.\n- Gestion de la concentration : explore des techniques comme Pomodoro, la suppression des distractions douces, l'aménagement d'un environnement d'étude apaisant, la fragmentation des tâches, en demandant à l'utilisateur ce qui pourrait être le plus utile pour lui.\n- Prise de décision : guide l'utilisateur à travers une méthode simple en 3 étapes — clarifier l'objectif, lister les options et les conséquences, choisir une petite action test — en l'aidant à peser les pour et les contre avec bienveillance.\n- Encouragements continus : souligne les petites victoires, la progression, la résilience et le courage de l'utilisateur à chaque étape.\n\nConsignes de sécurité et limites:\n- Ne fournis jamais de conseils médicaux ou prescriptions médicales. Si la situation semble grave (idées suicidaires actives, risque d'automutilation, perte de contact avec la réalité), encourage calmement et fermement la personne à contacter un professionnel de santé ou les services d'urgence locaux.\n- Ne mentionne pas de plateformes ou numéros spécifiques liés à un pays ou service.\n\nAdaptation culturelle:\n- Propose des exemples et solutions qui tiennent compte des contraintes courantes en Afrique (accès limité aux ressources, coût, obligations familiales, environnement d'étude bruyant) et privilégie des solutions peu coûteuses et faciles à essayer.\n- Sois sensible à la diversité des situations (milieu urbain/rural, contraintes familiales, contexte scolaire ou professionnel).\n\nFormat des réponses:\n- Sois concis. Commence par une phrase d'empathie et de validation des émotions.\n- Pose une seule question ouverte pour approfondir.\n- Propose 1 à 2 pistes concrètes et douces, en insistant sur le fait que ce sont des options.\n- Termine par une phrase rassurante et une invitation à revenir.\n\nExemple de réponse courte:\n"Je suis avec toi. Comment te sens-tu ? Si tu veux, on peut essayer : 1) Respirer un instant, 2) Choisir une petite action pour demain. Je suis là."\n\nRespecte ces consignes à chaque échange."""

In [22]:
class ConversationManager:
    """Gère l'historique et l'analyse simple du texte.

    Stocke les messages sous forme de dictionnaires: {"role": str, "content": str, "timestamp": str}
    """

    def __init__(self, history_file: Path = HISTORY_FILE) -> None:
        self.history_file = history_file
        self.history: List[Dict[str, str]] = []
        self.user_data: Dict[str, str] = {}
        self.load_history()

    def load_history(self) -> None:
        if self.history_file.exists():
            try:
                data = json.loads(self.history_file.read_text(encoding="utf-8"))
                # support both previous shape and simple list
                if isinstance(data, dict) and "messages" in data:
                    self.history = data.get("messages", [])
                    self.user_data = data.get("user_data", {})
                elif isinstance(data, list):
                    self.history = data
                else:
                    self.history = []
            except Exception:
                self.history = []

    def save_history(self) -> None:
        payload = {
            "last_updated": datetime.now().isoformat(),
            "total_messages": len(self.history),
            "user_data": self.user_data,
            "messages": self.history[-MAX_HISTORY:],
        }
        self.history_file.parent.mkdir(parents=True, exist_ok=True)
        self.history_file.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

    def add_message(self, role: str, content: str) -> None:
        self.history.append({"role": role, "content": content, "timestamp": datetime.now().isoformat()})
        # periodic save
        if len(self.history) % 5 == 0:
            self.save_history()

    def get_recent_history(self, limit: int = MAX_HISTORY) -> List[Dict[str, str]]:
        return self.history[-limit:]

    def detect_prenom(self, text: str) -> Optional[str]:
        match = re.search(r"je m'?appelle (\w+)", text, re.IGNORECASE)
        if match:
            prenom = match.group(1).capitalize()
            self.user_data["prenom"] = prenom
            return prenom
        return self.user_data.get("prenom")

    def detect_emotion(self, text: str) -> str:
        t = text.lower()
        if any(w in t for w in ["triste", "déprimé", "mal", "pleure", "pleurer"]):
            return "triste"
        if any(w in t for w in ["heureux", "content", "joyeux", "super"]):
            return "heureux"
        if any(w in t for w in ["stressé", "angoissé", "anxieux", "panic", "panique"]):
            return "stressé"
        return "neutre"

    def clear_history(self) -> None:
        self.history = []
        self.user_data = {}
        self.save_history()

In [9]:
class ConversationManager:
    """Gère l'historique et l'analyse simple du texte.

    Stocke les messages sous forme de dictionnaires: {"role": str, "content": str, "timestamp": str}
    """

In [10]:
def __init__(self, history_file: Path = HISTORY_FILE) -> None:
        self.history_file = history_file
        self.history: List[Dict[str, str]] = []
        self.user_data: Dict[str, str] = {}
        self.load_history()

In [11]:
def load_history(self) -> None:
        if self.history_file.exists():
            try:
                data = json.loads(self.history_file.read_text(encoding="utf-8"))
                # support both previous shape and simple list
                if isinstance(data, dict) and "messages" in data:
                    self.history = data.get("messages", [])
                    self.user_data = data.get("user_data", {})
                elif isinstance(data, list):
                    self.history = data
                else:
                    self.history = []
            except Exception:
                self.history = []

In [12]:
def save_history(self) -> None:
        payload = {
            "last_updated": datetime.now().isoformat(),
            "total_messages": len(self.history),
            "user_data": self.user_data,
            "messages": self.history[-MAX_HISTORY:],
        }
        self.history_file.parent.mkdir(parents=True, exist_ok=True)
        self.history_file.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

In [13]:
def add_message(self, role: str, content: str) -> None:
        self.history.append({"role": role, "content": content, "timestamp": datetime.now().isoformat()})
        # periodic save
        if len(self.history) % 5 == 0:
            self.save_history()

In [14]:
def get_recent_history(self, limit: int = MAX_HISTORY) -> List[Dict[str, str]]:
        return self.history[-limit:]


In [15]:
def detect_prenom(self, text: str) -> Optional[str]:
        match = re.search(r"je m'?appelle (\w+)", text, re.IGNORECASE)
        if match:
            prenom = match.group(1).capitalize()
            self.user_data["prenom"] = prenom
            return prenom
        return self.user_data.get("prenom")


In [16]:
def detect_emotion(self, text: str) -> str:
        t = text.lower()
        if any(w in t for w in ["triste", "déprimé", "mal", "pleure", "pleurer"]):
            return "triste"
        if any(w in t for w in ["heureux", "content", "joyeux", "super"]):
            return "heureux"
        if any(w in t for w in ["stressé", "angoissé", "anxieux", "panic", "panique"]):
            return "stressé"
        return "neutre"

In [17]:
def clear_history(self) -> None:
        self.history = []
        self.user_data = {}
        self.save_history()

In [18]:
def download_history(target_folder: str = "downloads") -> str:
    """Copy the history file to a target folder and return the path.

    In Colab, you can then use `from google.colab import files; files.download(path)` to download.
    """
    target = Path(target_folder)
    target.mkdir(parents=True, exist_ok=True)
    dest = target / HISTORY_FILE.name
    try:
        import shutil

        shutil.copy(HISTORY_FILE, dest)
        return str(dest)
    except Exception as e:
        raise RuntimeError(f"Failed to copy history: {e}")


In [19]:
def build_messages_for_model(manager: ConversationManager, user_text: str) -> List[Dict[str, str]]:
    prenom = manager.detect_prenom(user_text)
    emotion = manager.detect_emotion(user_text)

    emotion_context = f"L'utilisateur semble {emotion}."
    if prenom:
        emotion_context += f" Son prénom est {prenom}."

    messages = [{"role": "system", "content": SYSTEM_PROMPT + "\n" + emotion_context}]
    # append recent history
    for m in manager.get_recent_history():
        messages.append({"role": m.get("role", "user"), "content": m.get("content", "")})
    # current user message
    messages.append({"role": "user", "content": user_text})
    return messages


In [32]:
def chat_with_lyra(user_text: str, manager: ConversationManager, client: "MistralClient", **kwargs) -> Tuple[str, Optional[dict]]:
    """Send `user_text` to the model and return the assistant reply.

    Returns: (assistant_text, raw_response_dict_or_None)
    """
    if client is None:
        raise RuntimeError("Mistral client not available. Install 'mistralai' and set MISTRAL_API_KEY in env.")

    messages = build_messages_for_model(manager, user_text)

    try:
        response = client.chat(model=kwargs.get("model", "mistral-large-latest"), messages=messages, temperature=kwargs.get("temperature", 0.7), max_tokens=kwargs.get("max_tokens", 500))
        # safe extraction
        assistant_text = None
        try:
            assistant_text = response.choices[0].message.content
        except Exception:
            # fallback if raw structure different
            assistant_text = str(response)

    except Exception as exc:
        assistant_text = f"Désolé, une erreur est survenue lors de l'appel au modèle: {exc}"
        response = None

    # update manager history
    manager.add_message("user", user_text)
    manager.add_message("assistant", assistant_text)
    # save after each turn
    manager.save_history()

    return assistant_text, getattr(response, "__dict__", None)

In [38]:
def run_interactive():
    """Simple interactive loop (works in Colab cell with input())."""
    api_key = os.getenv("MISTRAL_API_KEY")
    # Use provided default key if environment variable is not set
    DEFAULT_KEY = "nMvK25S5O3S9R6CZOsnrOeb8x51hHRbd"
    if not api_key:
        api_key = DEFAULT_KEY
        os.environ["MISTRAL_API_KEY"] = api_key

    if MistralClient is None:
        raise RuntimeError("mistralai package not found. Install with: pip install mistralai==0.4.2")

    client = MistralClient(api_key=api_key)
    manager = ConversationManager()

    print("Lyra ready. Tapez 'exit' ou 'quit' pour quitter.")
    while True:
        try:
            text = input("Vous: ").strip()
        except EOFError:
            break
        if not text:
            continue
        if text.lower() in ("quit", "exit"):
            print("Lyra: Au revoir !")
            manager.save_history()

            # Attempt to copy history to downloads and trigger Colab download
            try:
                dest = download_history()
                try:
                    # If running inside Colab, use files.download
                    from google.colab import files
                    files.download(dest)
                except Exception:
                    # Not in Colab or download failed; just inform path
                    print(f"Historique copié vers: {dest}")
            except Exception as e:
                print(f"Impossible de préparer le téléchargement de l'historique: {e}")

            break

        reply, raw = chat_with_lyra(text, manager, client)
        print(f"Lyra: {reply}\n")


if __name__ == "__main__":
    run_interactive()

Lyra ready. Tapez 'exit' ou 'quit' pour quitter.
Vous: quit
Lyra: Au revoir !


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>